In [ ]:
import json
import pandas as pd
from elasticsearch import Elasticsearch
import mariadb

# **1-POSTGRESQL**

# **1-MARIADB**

# **2-ELASTICSEARCH**

In [ ]:
es = Elasticsearch("http://localhost:9200")

res = es.search(index='my_index', body={
     'query': {
         'range': {
             'position': {
                 'gte': 1,
                 'lte': 3
             }
         }
     },
     'size': 1
})

# **3-GRAPHDB**